In [ ]:
# import os
# os.system("pip install --upgrade pip")
# os.system("python3.8 -m pip install --upgrade pip")
# os.system("pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis --upgrade")
# os.system("pip install pycelonis")

from pycelonis import *
from pycelonis.pql import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, balanced_accuracy_score

url = "https://academic-if21b054-technikum-wien-at.eu-2.celonis.cloud/"

file = open('./Machine_Learning/APP_KEY.txt')
# file = open('APP_KEY.txt')
api_token = file.read()
file.close()

key_type = "APP_KEY"
celonis = get_celonis(base_url=url, api_token=api_token, key_type=key_type, permissions=False)

data_pool_name = "datapool-filtered-final-version"
data_model_name = "datamodel-filtered-final-version"
activity_table_name = "activities"
rating_table_name = "rating"

data_pool = celonis.data_integration.get_data_pools().find(data_pool_name)
data_model = data_pool.get_data_models().find(data_model_name)
activity_table = data_model.get_tables().find(activity_table_name)
rating_table = data_model.get_tables().find(rating_table_name)

query = PQL(distinct=False, limit=None, offset=None)
query += PQLColumn(name="Zeit", query=""" "ACTIVITIES"."zeit" """)
query += PQLColumn(name="Vollständiger Name", query=""" "ACTIVITIES"."Vollständiger Name" """)
query += PQLColumn(name="Ereigniskontext - Ereignisname", query=""" "ACTIVITIES"."Ereigniskontext - Ereignisname" """)

activity_df = data_model.export_data_frame(query)

query = PQL(distinct=False, limit=None, offset=None)
query += PQLColumn(name="Nr", query=""" "RATING"."Nr" """)
query += PQLColumn(name="Test: Quiz 1 (Punkte)", query=""" "RATING"."Test: Quiz 1 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 2 (Punkte)", query=""" "RATING"."Test: Quiz 2 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 3 (Punkte)", query=""" "RATING"."Test: Quiz 3 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 4 (Punkte)", query=""" "RATING"."Test: Quiz 4 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 5 (Punkte)", query=""" "RATING"."Test: Quiz 5 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 6 (Punkte)", query=""" "RATING"."Test: Quiz 6 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 7 (Punkte)", query=""" "RATING"."Test: Quiz 7 (Punkte)" """)
query += PQLColumn(name="Quiz gesamt (Punkte)", query=""" "RATING"."Quiz gesamt (Punkte)" """)
query += PQLColumn(name="Test: Exam Attempt 1 (Punkte)", query=""" "RATING"."Test: Exam Attempt 1 (Punkte)" """)
query += PQLColumn(name="Kurs gesamt (Punkte)", query=""" "RATING"."Kurs gesamt (Punkte)" """)
query += PQLColumn(name="Name", query=""" "RATING"."Name" """)

rating_df = data_model.export_data_frame(query)

################################################# DECISION TREES #################################################

df = rating_df
# extract X,y arrays
quiz_columns = [string for string in df.columns if "Quiz" in string and "gesamt" not in string]
X = df[quiz_columns].astype(float)
y_test = df["Test: Exam Attempt 1 (Punkte)"].astype(float)
y_total = df["Kurs gesamt (Punkte)"].astype(float)

mmSc = MinMaxScaler()
mmSc.fit(X)
X = mmSc.transform(X)
X = pd.DataFrame(X)
y = y_total

errors = []
for i in range(100):
    collector = []
    # use leave one out, because dataset is so small
    loo = LeaveOneOut()

    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        tree_reg = DecisionTreeRegressor().fit(X_train, y_train)
        y_hat = tree_reg.predict(X_test)
        collector.append(y_hat[0])

    y_pred = np.round(np.array(collector), 3)   

    errors.append({'idx': i,
                  'mae': mean_absolute_error(y, y_pred),
                  'rmse': mean_squared_error(y, y_pred, squared=False),
                  "pred": np.where(y_pred < 50, 0, 1),
                  "proba": y_pred})
    
error_df = pd.DataFrame(errors).round(3)
pred_tree = error_df['pred'].iloc[error_df['mae'].idxmax()]
points_tree = error_df['proba'].iloc[error_df['mae'].idxmax()]

################################################# ADD DECISION TREES RESULT TO RESULT DATA FRAME #################################################

result_df = rating_df
result_df = result_df.sort_values('Nr')
result_df['Abschluss_tree'] = pred_tree
result_df['Punkte_tree'] = points_tree
result_df = result_df.round(2)

data_pool.create_table(df=result_df, table_name="DecisionTrees", drop_if_exists=True, force=True)

try:
    data_model.get_tables().find("DecisionTrees").delete()
except:
    print("table does not exist yet")

data_model.add_table(name="DecisionTrees", alias="DecisionTrees")

data_model.reload()